# Chatea con modelos de visión

**Si estás buscando la aplicación web, revisa la carpeta src/.**

Este notebook solo se proporciona para experimentar manualmente con el modelo de visión.

## Autentícate en OpenAI

El siguiente código se conecta a OpenAI, ya sea usando una cuenta de Azure OpenAI, modelos de GitHub, o modelo local de Ollama. Mira el README para instrucciones sobre cómo configurar el archivo `.env`.

In [1]:
import os
import openai

# GitHub Models client configuration
print("Usando GitHub Models con GITHUB_TOKEN como clave")
openai_client = openai.OpenAI(
    api_key=os.environ["GITHUB_TOKEN"],
    base_url="https://models.github.ai/inference",
)
model_name = os.getenv("GITHUB_MODEL", "openai/gpt-4o")

print(f"Usando modelo {model_name}")

Usando GitHub Models con GITHUB_TOKEN como clave
Usando modelo openai/gpt-4o


## Envía una imagen por URL

In [2]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "¿es un unicornio?", "type": "text"},
            {
                "image_url": {"url": "https://upload.wikimedia.org/wikipedia/commons/6/6e/Ur-painting.jpg"},
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=model_name, messages=messages)

print(response.choices[0].message.content)

No, no es un unicornio. Es un toro con dos cuernos, como los típicos de esta especie. Los unicornios, según la mitología, son criaturas con un solo cuerno en la frente.


## Envía una imagen por Data URI



In [3]:
import base64


def open_image_as_base64(filename):
    with open(filename, "rb") as image_file:
        image_data = image_file.read()
    image_base64 = base64.b64encode(image_data).decode("utf-8")
    return f"data:image/png;base64,{image_base64}"

In [7]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿estos son cocodrilos o caimanes?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/mystery_reptile.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

En la imagen se observan **cocodrilos**, probablemente cocodrilos del género *Crocodylus*. Aunque puede ser difícil diferenciarlos de los caimanes solo por la imagen, los cocodrilos suelen tener un hocico más largo y puntiagudo, con dientes visibles incluso cuando la boca está cerrada, características que parecen coincidir con los animales en la fotografía.


## Casos de uso para análisis de imágenes

### Accesibilidad

#### Asistencia para personas con discapacidad visual

In [9]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿hay algo bueno para veganos en este menú?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/menu.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

El menú no parece tener opciones específicamente etiquetadas como veganas. Sin embargo, podrías considerar algunas opciones modificadas o preguntar en el restaurante si pueden adaptarlas. Aquí te dejo algunas ideas:

1. **Spinaci Soffritti (Antipasti)**: Las espinacas salteadas con limón y ajo podrían ser veganas, pero sería bueno confirmar si no usan mantequilla u otros ingredientes no veganos.

2. **Insalata di Mista (Zuppe & Insalate)**: Una ensalada de verdes mixtos con vinagreta podría ser vegana si confirmas que la vinagreta no contiene ingredientes no veganos como miel.

3. **Panzanella con Fagioli (Zuppe & Insalate)**: La ensalada de pan con frijoles y vegetales podría ser apta para veganos si no incluye derivados lácteos o si se elimina el queso y verificas el pan.

4. **Zuppa di Cavolo (Zuppe & Insalate)**: La crema de coliflor gratinada podría adaptarse, pero es probable que incluyan mantequilla o crema. Pregunta si es posible modificarla.

Para asegurarte, lo mejor sería pr

#### Subtítulos automáticos de imágenes

In [12]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "Sugiere un texto alternativo para esta imagen", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/azure_arch.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

Diagrama que representa una arquitectura en Azure para una aplicación de contenedor. En el centro está el "Container App" que se conecta hacia varias entidades:  
1. **Container Apps Environment** (entorno de aplicaciones de contenedor) que alberga la aplicación.  
2. **Azure AI Services** (servicios de inteligencia artificial).  
3. **Managed Identity** (identidad administrada) para el manejo de permisos seguros.  
4. **Log Analytics Workspace** (espacio de trabajo de análisis de logs) para monitoreo y generación de registros.  
5. **Container Registry** (registro de contenedores), para almacenar imágenes de contenedores.  
6. **Key Vault** (bóveda de claves) para gestionar secretos y claves de acceso.  

El flujo muestra cómo estos componentes están relacionados para crear una solución funcional basada en contenedores, utilizando herramientas nativas de Azure. 


### Automatización de procesos de negocio

#### Procesamiento de reclamaciones de seguros

In [13]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "system",
            "content": (
                "Eres un asistente de IA que ayuda a las compañías de seguros de autos a procesar reclamaciones. "
                "Aceptas imágenes de autos dañados que se envían con las reclamaciones, y puedes hacer juicios "
                "sobre las causas del daño automovilístico y la validez de las reclamaciones sobre ese daño."
            ),
        },
        {
            "role": "user",
            "content": [
                {"text": "La reclamación dice que este daño es por granizo. ¿Es válida?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/dented_car.jpg")}, "type": "image_url"},
            ],
        },
    ],
)

print(response.choices[0].message.content)

No, esta reclamación no es válida si se atribuye a daños por granizo. Los daños ocasionados por granizo suelen consistir en múltiples pequeñas abolladuras en la superficie del vehículo, principalmente en el techo, el capó y el maletero, y no causan una deformación severa o una destrucción extensa de la carrocería como se observa en la imagen. El daño en esta foto es más consistente con un impacto frontal, posiblemente con otro vehículo u objeto grande.


#### Análisis de gráficos

In [14]:
messages = [
    {
        "role": "user",
        "content": [
            {"text": "¿En qué zona estamos perdiendo más árboles?", "type": "text"},
            {
                "image_url": {
                    "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg/1280px-20210331_Global_tree_cover_loss_-_World_Resources_Institute.svg.png"
                },
                "type": "image_url",
            },
        ],
    }
]
response = openai_client.chat.completions.create(model=model_name, messages=messages)

print(response.choices[0].message.content)

Según el gráfico, la zona tropical está perdiendo más árboles. Esto se puede observar en las barras verdes oscuras, que tienen las pérdidas más significativas en comparación con las otras zonas climáticas (boreal, templada y subtropical).


#### Análisis de tablas

In [15]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿Cuál es la planta más barata?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/page_0.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

La planta más barata en esta lista es **Agrostis pallens (Thingrass)**, con un precio de **$0.58**.


#### Soporte de electrodomésticos

In [16]:
response = openai_client.chat.completions.create(
    model=model_name,
    messages=[
        {
            "role": "user",
            "content": [
                {"text": "¿Cómo configuro esto para lavar los platos rápidamente?", "type": "text"},
                {"image_url": {"url": open_image_as_base64("img/dishwasher.png")}, "type": "image_url"},
            ],
        }
    ],
)

print(response.choices[0].message.content)

Si deseas lavar los platos rápidamente con este lavavajillas, sigue estos pasos:

1. **Enciende el lavavajillas** presionando el botón de encendido/apagado ("On/Off").
2. Selecciona el programa llamado **"Quick 45°"** (tercer botón desde la derecha en la fila superior). Este programa está diseñado para un lavado rápido a 45 °C.
3. Opcionalmente, si tienes menos cantidad de platos, puedes presionar el botón **"Half Load"** (segundo botón desde la derecha en la fila inferior) para optimizar agua y energía.
4. Si deseas acelerar aún más el ciclo, también puedes presionar el botón **"Vario Speed"** (segundo botón desde la derecha en la fila inferior). Esto reduce el tiempo total de lavado, aunque puede consumir más agua y energía.
5. Una vez configurado, presiona **"Start"** (botón de la esquina inferior derecha) para iniciar el ciclo.

¡Y listo! Tendrás tus platos lavados rápidamente.
